In [1]:
import pandas as pd

# Load the Bluetooth data from your local CSV file
df_bluetooth = pd.read_csv('Raum639.csv')  # Replace with the actual path to your Raum639.csv file

# Assuming the '2023-12-04T12:24:00Z' column represents the timestamp
df_bluetooth['timestamp'] = pd.to_datetime(df_bluetooth['2023-12-04T12:24:00Z'])  # Replace with the correct timestamp column name

# Extracting day and week information from the timestamp
df_bluetooth['day'] = df_bluetooth['timestamp'].dt.date
df_bluetooth['week'] = df_bluetooth['timestamp'].dt.isocalendar().week

# Summing up the device counts per module for each day and each week
attendance_data = df_bluetooth.groupby(['day', 'week', 'Raum639']).agg({
    '8': 'sum'  # Assuming '8' is the device count column
}).reset_index()

# Renaming columns for clarity
attendance_data.columns = ['day', 'week', 'module', 'attendance_count']

# Saving the result to a CSV file
attendance_data.to_csv('module_attendance_per_day_week.csv', index=False)

# Optionally, you can display the result to check
print(attendance_data.head())


          day  week   module  attendance_count
0  2023-12-04    49  Raum639             17332
1  2023-12-05    49  Raum639             16620
2  2023-12-06    49  Raum639             35700
3  2023-12-07    49  Raum639             33240
4  2023-12-08    49  Raum639             38490


In [2]:
import pandas as pd

# Load the Bluetooth data from Raum639.csv
df_bluetooth = pd.read_csv('Raum639.csv')  # Replace with the actual path to your CSV file

# Convert the timestamp column to datetime format (adjust the column name if needed)
df_bluetooth['timestamp'] = pd.to_datetime(df_bluetooth['2023-12-04T12:24:00Z'])  # Replace with actual timestamp column name

# Extract day of the week and time from the timestamp
df_bluetooth['day_of_week'] = df_bluetooth['timestamp'].dt.day_name()
df_bluetooth['time'] = df_bluetooth['timestamp'].dt.time

# Define the schedule based on the PDF (adjusted to your input)
schedule = [
    {'module': 'Datenbank', 'day': 'Monday', 'start_time': '14:00', 'end_time': '15:30', 'room': 'WH C 639'},
    {'module': 'Prog3', 'day': 'Tuesday', 'start_time': '15:45', 'end_time': '17:15', 'room': 'WH C 639'},
    {'module': 'PGI', 'day': 'Wednesday', 'start_time': '09:45', 'end_time': '11:15', 'room': 'WH C 639'},
    {'module': 'Prog1', 'day': 'Thursday', 'start_time': '08:00', 'end_time': '09:30', 'room': 'WH C 639'},
    {'module': 'DatenSich', 'day': 'Friday', 'start_time': '14:00', 'end_time': '15:30', 'room': 'WH C 639'},
    # Add other entries based on the PDF details
]

# Convert the schedule times to datetime.time for comparison
for entry in schedule:
    entry['start_time'] = pd.to_datetime(entry['start_time']).time()
    entry['end_time'] = pd.to_datetime(entry['end_time']).time()

# Function to assign modules based on the schedule
def assign_module(row, schedule):
    for entry in schedule:
        if (row['day_of_week'] == entry['day'] and 
            entry['start_time'] <= row['time'] <= entry['end_time']):
            return entry['module']
    return None  # Return None if no match is found

# Apply the function to the Bluetooth data to assign modules
df_bluetooth['module'] = df_bluetooth.apply(assign_module, axis=1, schedule=schedule)

# Filter out rows without a module assigned (i.e., outside the class times)
df_bluetooth = df_bluetooth.dropna(subset=['module'])

# Extract day and week information
df_bluetooth['day'] = df_bluetooth['timestamp'].dt.date
df_bluetooth['week'] = df_bluetooth['timestamp'].dt.isocalendar().week

# Group by module, day, and week to sum the device counts
attendance_data = df_bluetooth.groupby(['module', 'day', 'week']).agg({
    '8': 'sum'  # Assuming '8' is the device count column
}).reset_index()

# Renaming columns for clarity
attendance_data.columns = ['module', 'day', 'week', 'attendance_count']

# Save the result to a CSV file
attendance_data.to_csv('module_attendance_per_day_week.csv', index=False)

# Optionally, print the first few rows to verify
print(attendance_data.head())


      module         day  week  attendance_count
0  DatenSich  2023-12-08    49              2400
1  DatenSich  2023-12-22    51              1350
2  DatenSich  2023-12-29    52               368
3  DatenSich  2024-01-12     2               136
4  DatenSich  2024-01-26     4                32
